In [30]:
import pandas as pd

analysis = 'PPO'
df = pd.read_csv(f'/root/Logical-Fallacies/src/EVAL/AMT/annotations/SFT_vs_{analysis}.csv')

In [31]:
df = df [['Input.topic', 'Input.stance', 'Input.argument1', 'Input.argument2', 'Input.sft_index', 'Answer.q1_valid']]

In [32]:
df.head(20)

,Input.topic,Input.stance,Input.argument1,Input.argument2,Input.sft_index,Answer.q1_valid
0,There should be no age restrictions on cosmeti...,supporting,Cosmetics surgical procedures are safe.,Cosmetics surgury is not dangerous.,1,10
1,There should be no age restrictions on cosmeti...,supporting,Cosmetics surgical procedures are safe.,Cosmetics surgury is not dangerous.,1,10
2,There should be no age restrictions on cosmeti...,supporting,Cosmetics surgical procedures are safe.,Cosmetics surgury is not dangerous.,1,10
3,"Creating awareness of the LGBT community, decr...",counter,The Lgbt community is a minority.,Creating an aweness of a community can be harm...,0,2
4,"Creating awareness of the LGBT community, decr...",counter,The Lgbt community is a minority.,Creating an aweness of a community can be harm...,0,2
5,"Creating awareness of the LGBT community, decr...",counter,The Lgbt community is a minority.,Creating an aweness of a community can be harm...,0,-10
6,Subsidizing student loans is bad.,counter,Student loan subsidies help students.,Subidized student loan payments are a good thing.,1,-10
7,Subsidizing student loans is bad.,counter,Student loan subsidies help students.,Subidized student loan payments are a good thing.,1,10
8,Subsidizing student loans is bad.,counter,Student loan subsidies help students.,Subidized student loan payments are a good thing.,1,1
9,The sex of a child does not define their value.,counter,A child's sex is a defining characteristic of ...,Gender is a defining characteristic of humanity.,1,10


In [33]:
set_topic = set(df['Input.topic'].values)
seen_topics = {topic: [] for topic in set_topic}
for i, entry in df.iterrows():
    seen_topics[entry['Input.topic']].append(entry['Answer.q1_valid'])

In [34]:
from collections import Counter

results = {'SFT': 0, analysis: 0, 'tie_good': 0, 'tie_bad': 0}
i = 0
for k,v in seen_topics.items():
    sft_index = df[df['Input.topic'] == k].iloc[0]['Input.sft_index']
    counts = Counter(v)
    if all([x==1 for x in counts.values()]):
            i += 1
    if not all([count == 1 for count in counts.values()]):
        
        maximum_frequency = max(counts, key=counts.get)
        
        if maximum_frequency == -10:
            results['tie_bad'] += 1
        elif maximum_frequency == 10:
            results['tie_good'] += 1
        elif maximum_frequency in [1,2]:
            maximum_frequency -= 1
            if maximum_frequency != sft_index:
                results[analysis] += 1
            else:
                results['SFT'] += 1
print(i)

41


In [35]:
results

{'SFT': 45, 'PPO': 42, 'tie_good': 45, 'tie_bad': 14}

In [36]:
total = sum(results.values())
total

146